In [2]:
import pandas as pd
import numpy as np

In [19]:
evaluation_df = pd.read_csv('../EvaluationDic/EN-FR-Matthias_Buchmeier_Dic.csv', delimiter='#', error_bad_lines=False)
build_df_word2vec = pd.read_csv('../BuildDictionary/dict.tsv', delimiter='\t', error_bad_lines=False,
                                names=["word", "trans", "pos", 'confidence'])
# build_df_glove = pd.read_csv('../BuildDictionary/TIADbaseline_glove_trans_en-fr.tsv', delimiter='\t', error_bad_lines=False,
#                              names=["word", "trans", "pos", 'confidence'])

In [20]:
build_df_word2vec

,word,trans,pos,confidence
0,positioner,propriété rurale,noun,0.90
1,by a whisker,dramaturgiquement,adverb,0.73
2,nice,ravi,adjective,0.98
3,orthogonalisation,thiocyanate,noun,0.89
4,help,aider,verb,0.99
...,...,...,...,...
59572,surly,malséant,adjective,0.90
59573,legal project,barysphère,noun,0.91
59574,it never rains but it pours,d'accord,interjection,0.85
59575,shouting,de fil en aiguille,adverb,0.77


In [21]:
print(build_df_word2vec.columns)
# print(build_df_glove.columns)

Index(['word', 'trans', 'pos', 'confidence'], dtype='object')


In [27]:
evaluation_df = evaluation_df.drop(['sens'], axis=1)
evaluation_df = evaluation_df.drop_duplicates()
evaluation_df.count()

word     108063
trans    108065
pos      108067
dtype: int64

In [23]:
evaluation_df.columns

Index(['word', 'sens', 'trans', 'pos'], dtype='object')

In [24]:
build_df_word2vec = build_df_word2vec.drop(['confidence'], axis=1)
# build_df_word2vec = build_df_word2vec.drop_duplicates()
build_df_word2vec.count()

word     59575
trans    59521
pos      59522
dtype: int64

In [35]:
build_df_word2vec.loc[build_df_word2vec['word']==build_df_word2vec['trans']]

,word,trans,pos
5,Isabela,Isabela,properNoun
6,Pujol,Pujol,properNoun
8,Cecelia,Cecelia,properNoun
11,Silverman,Silverman,properNoun
15,Gabarron,Gabarron,properNoun
...,...,...,...
59009,massif,massif,noun
59196,Ermua,Ermua,properNoun
59197,PSE,PSE,noun
59388,Elgoibar,Elgoibar,properNoun


In [29]:
df_all = build_df_word2vec.merge(evaluation_df,
                   how='left', indicator=True)
df_all

,word,trans,pos,_merge
0,positioner,propriété rurale,noun,left_only
1,by a whisker,dramaturgiquement,adverb,left_only
2,nice,ravi,adjective,left_only
3,orthogonalisation,thiocyanate,noun,left_only
4,help,aider,verb,both
...,...,...,...,...
59572,surly,malséant,adjective,left_only
59573,legal project,barysphère,noun,left_only
59574,it never rains but it pours,d'accord,interjection,left_only
59575,shouting,de fil en aiguille,adverb,left_only


In [30]:
a = df_all.loc[df_all['_merge']=='both']['word'].count()
a

12020

In [ ]:
df.loc[df]

In [34]:
df_all.loc[df_all['_merge']=='left_only'][['word', 'trans', 'pos']].to_csv(path_or_buf='../BuildDictionary/check.csv', 
                                                                          sep='\t',index=False)

In [31]:
df_all_word_pos = build_df_word2vec[['word','pos']].merge(evaluation_df[['word','pos']],
                   how='left', indicator=True)
df_all_word_pos

,word,pos,_merge
0,positioner,noun,left_only
1,by a whisker,adverb,left_only
2,nice,adjective,both
3,nice,adjective,both
4,nice,adjective,both
...,...,...,...
82666,surly,adjective,both
82667,legal project,noun,left_only
82668,it never rains but it pours,interjection,left_only
82669,shouting,adverb,left_only


In [32]:
b = df_all_word_pos.loc[df_all_word_pos['_merge']=='both']['word'].count()
b

44987

In [60]:
def get_accuracy(df_build, df_evaluation):
    df_all = df_build.merge(df_evaluation.drop_duplicates(),
                   how='left', indicator=True)
    a = df_all.loc[df_all['_merge']=='both']['word'].count()

    df_all_word_pos = df_build[['word','pos']].merge(df_evaluation[['word','pos']],
                   how='left', indicator=True)
    b = df_all_word_pos.loc[df_all_word_pos['_merge']=='both']['word'].count()
    return a/b

In [62]:
print(get_accuracy(build_df_word2vec, evaluation_df))
print(get_accuracy(build_df_glove, evaluation_df))

0.20889147174197298
0.10804548723001944
